In [1]:
import pandas as pd

In [2]:
posts_df = pd.read_csv("C:\\Users\\Dell\\OneDrive - Northeastern University\\courses\\big data and intl analytics\\DAMG7245-Summer2023\\final project\\dataset_converted\\3d_printing\\posts.csv")

In [3]:
comments_df = pd.read_csv("C:\\Users\\Dell\\OneDrive - Northeastern University\\courses\\big data and intl analytics\\DAMG7245-Summer2023\\final project\\dataset_converted\\3d_printing\\comments.csv")

In [4]:
posts_df.head()

,Unnamed: 0,Id,PostTypeId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastActivityDate,Title,Tags,AnswerCount,CommentCount,AcceptedAnswerId,LastEditorUserId,ParentId
0,0,1,1,2016-01-12T18:45:19.963,10,424.0,<p>When I've printed an object I've had to cho...,16.0,2017-10-31T02:31:08.560,How to obtain high resolution prints in a shor...,<resolution><speed><quality>,2.0,6,51.0,NaN,NaN
1,1,2,1,2016-01-12T18:45:51.287,34,7377.0,"<p>I would like to buy a 3D printer, but I'm c...",20.0,2019-06-10T23:18:34.190,Is 3D printing safe for your health?,<print-material><safety><health>,4.0,1,12.0,334.0,NaN
2,2,3,1,2016-01-12T18:46:22.083,18,2678.0,<p>I know the minimum layer height will effect...,11.0,2016-09-19T15:41:06.537,How important is the minimum layer height on a...,<quality><resolution>,3.0,5,152.0,11.0,NaN
3,3,4,1,2016-01-12T18:50:55.973,18,384.0,<p>Plastic is used in 3D FDM/FFF printing part...,16.0,2016-06-10T13:32:20.493,Are there any metals that exhibit a large glas...,<fdm><material><print-material><metal-parts>,4.0,0,1289.0,98.0,NaN
4,4,5,1,2016-01-12T18:53:53.623,40,3985.0,<p>What are the main differences when using AB...,11.0,2017-08-02T09:49:07.263,How is PLA different from ABS material?,<filament><abs><fdm><pla>,5.0,5,77.0,20.0,NaN


In [5]:
comments_df.head()

,Unnamed: 0,Id,PostId,Score,Text,CreationDate,UserId,ContentLicense
0,0,1,1,4,Did I just place the first upvote? Congrats o...,2016-01-12T18:47:12.573,23.0,CC BY-SA 3.0
1,1,2,3,1,I think it would be a good idea to specify wha...,2016-01-12T19:01:35.180,10.0,CC BY-SA 3.0
2,2,3,1,1,What are you looking for in an answer? You are...,2016-01-12T19:42:08.613,26.0,CC BY-SA 3.0
3,3,4,21,9,It is worth noting here that acetone finishing...,2016-01-12T19:42:19.530,36.0,CC BY-SA 3.0
4,4,5,5,7,I think this is too broad for a good exhaustiv...,2016-01-12T19:44:02.667,36.0,CC BY-SA 3.0


In [6]:
posts_df.dtypes

Unnamed: 0            int64
Id                    int64
PostTypeId            int64
CreationDate         object
Score                 int64
ViewCount           float64
Body                 object
OwnerUserId         float64
LastActivityDate     object
Title                object
Tags                 object
AnswerCount         float64
CommentCount          int64
AcceptedAnswerId    float64
LastEditorUserId    float64
ParentId            float64
dtype: object

In [7]:
comments_df.dtypes

Unnamed: 0          int64
Id                  int64
PostId              int64
Score               int64
Text               object
CreationDate       object
UserId            float64
ContentLicense     object
dtype: object

In [8]:
posts_df.shape

(14475, 16)

In [9]:
comments_df.shape

(24320, 8)

In [10]:
# Rename columns in posts_df
posts_df = posts_df.rename(columns={
    'Id': 'post_id',
    'PostTypeId': 'posttypeid',
    'CreationDate': 'post_creationdate',
    'Score': 'post_score',
    'ViewCount': 'post_viewcount',
    'Body': 'post_body',
    'Title': 'post_title',
    'Tags': 'post_tags',
    'AcceptedAnswerId': 'post_acceptedanswerid'
})

# Rename columns in comments_df
comments_df = comments_df.rename(columns={
    'Text': 'comments_text',
    'Score': 'comments_score',
    'CreationDate': 'comments_creationdate'
})




In [11]:
# Join posts dataframe with itself on post_acceptedanswerid and post_id
posts_df = pd.merge(posts_df, posts_df, left_on='post_acceptedanswerid', right_on='post_id', how='left', suffixes=('', '_accepted'))

# Keep only the required columns
posts_df = posts_df[['post_id', 'posttypeid', 'post_creationdate', 'post_score', 
               'post_viewcount', 'post_body', 'post_title', 
               'post_tags',
               'post_body_accepted', 'post_score_accepted']]


In [12]:
# Keep only rows where posttypeid is 1 or 2
posts_df = posts_df[posts_df['posttypeid'].isin([1, 2])]

In [13]:
# Keep only rows where post_tags is <2d>, <3d-design>, or null
mask = posts_df['post_tags'].isin(['<2d>', '<3d-design>']) | posts_df['post_tags'].isnull()
posts_df = posts_df[mask]

In [14]:
posts_df.shape

(8508, 10)

In [15]:
# Keep only the first 100 rows of the DataFrame
posts_df = posts_df.head(100)
posts_df.shape

(100, 10)

In [16]:
# Merge the dataframes
df_result = pd.merge(posts_df, comments_df, left_on='post_id', right_on='PostId', how='left')

In [17]:
# Remove rows where comments_text, comments_score, or comments_creationdate is null
mask = df_result['comments_text'].notnull() & df_result['comments_score'].notnull() & df_result['comments_creationdate'].notnull()
df_result = df_result[mask]

In [18]:
# Keep only the required columns
df_result = df_result[['post_id', 'posttypeid', 'post_creationdate', 'post_score', 
                       'post_viewcount', 'post_body', 'post_title', 
                       'post_tags', 'post_body_accepted', 'post_score_accepted', 
                       'comments_text', 'comments_score', 
                       'comments_creationdate']]

In [19]:
# Create a posts dataframe
posts = df_result[['post_id', 'posttypeid', 'post_creationdate', 'post_score', 
                   'post_viewcount', 'post_body', 'post_title', 
                   'post_tags', 'post_body_accepted', 'post_score_accepted']].drop_duplicates()



In [20]:
# Create a comments dataframe
comments = df_result[['post_id', 'comments_text', 
                      'comments_score', 
                      'comments_creationdate']]

In [21]:
posts.shape

(54, 10)

In [22]:
comments.shape

(136, 4)

In [23]:
posts.to_csv('posts_cleaned.csv', index=False)

In [24]:
comments.to_csv('comments_cleaned.csv', index=False)

In [25]:
posts.head()

,post_id,posttypeid,post_creationdate,post_score,post_viewcount,post_body,post_title,post_tags,post_body_accepted,post_score_accepted
0,9,2,2016-01-12T18:58:04.410,18,NaN,<p>Almost all 3D printers have issues that cou...,NaN,NaN,NaN,NaN
2,12,2,2016-01-12T19:13:00.710,23,NaN,<p>There is very little information about safe...,NaN,NaN,NaN,NaN
4,15,2,2016-01-12T19:25:07.817,5,NaN,<p>Apart from the inherent process itself and ...,NaN,NaN,NaN,NaN
6,17,2,2016-01-12T19:27:20.113,5,NaN,"<p>I""m no expert on this, but the article at <...",NaN,NaN,NaN,NaN
8,18,2,2016-01-12T19:28:04.263,10,NaN,<p>PET(G) is a strong contender. It is very st...,NaN,NaN,NaN,NaN


In [26]:
comments.head()

,post_id,comments_text,comments_score,comments_creationdate
0,9,I'm not sure this applies worldwide: but in Ca...,0.0,2016-01-13T01:14:36.707
1,9,"""*May* not be healthy"" can't be stressed out e...",1.0,2016-02-07T10:51:22.563
2,12,I think I'm going to start turning on the vent...,0.0,2016-01-13T17:28:35.820
3,12,Great answer. In the molecular level there is ...,5.0,2016-02-07T10:42:09.990
4,15,"Are these really health issues? Is ""you might ...",1.0,2016-01-14T06:56:34.723


In [27]:
posts.dtypes

post_id                  int64
posttypeid               int64
post_creationdate       object
post_score               int64
post_viewcount         float64
post_body               object
post_title              object
post_tags               object
post_body_accepted      object
post_score_accepted    float64
dtype: object

In [28]:
comments.dtypes

post_id                    int64
comments_text             object
comments_score           float64
comments_creationdate     object
dtype: object

In [ ]:
#pip install openai
from scipy.spatial.distance import cdist
import openai
import pandas as pd

In [ ]:
openai.api_key = ""

In [ ]:
df = pd.read_csv("C:\\Users\\Dell\\OneDrive - Northeastern University\\courses\\big data and intl analytics\\DAMG7245-Summer2023\\final project\\posts_cleaned.csv")

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
#openai embedding using text-embedding model
def get_openai_embedding(text):
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response['data'][0]['embedding']


In [ ]:
#funcction to combine the text needed to be embeded
def get_combined_text(row):
    title = row['post_title']
    post_body = row['post_body']
    comments = row['comments']
    tags = row['tags']
    combined_text = f"{title} {post_body} {comments} {tags}"
    return combined_text


In [ ]:
#creating and appending the embedding to df
df['openai_embeddings'] = df.apply(get_combined_text, axis=1).apply(get_openai_embedding)

In [ ]:
df.head()

In [ ]:
df.to_csv('demo2.csv', index=False)

In [ ]:
user_input="3d printer strength"

In [ ]:
#euclidian distance - similarity search
def get_most_similar_title(user_input, df):
    user_input_embedding = get_openai_embedding(user_input)
    distances = cdist([user_input_embedding], df['openai_embeddings'].tolist(), metric='euclidean')
    most_similar_index = distances.argmin()
    most_similar_title = df.iloc[most_similar_index]['title']
    return most_similar_title

In [ ]:
most_similar_title = get_most_similar_title(user_input, df)
print(f"The most similar title is: {most_similar_title}")